In [46]:
using Pkg
Pkg.activate("../Project.toml")

using ITensors
import NDTensors
include("summary.jl")
include("utils_precise.jl")

  Activating project at `~/Documents/QML Project/QuantumInspiredML`


transformData (generic function with 2 methods)

In [26]:
function GenerateStartingMPS(χ_init, site_indices::Vector{Index{Int64}};
    num_classes = 2, random_state=nothing)
    """Generate the starting weight MPS, W using values sampled from a 
    Gaussian (normal) distribution. Accepts a χ_init parameter which
    specifies the initial (uniform) bond dimension of the MPS."""
    
    if random_state !== nothing
        # use seed if specified
        Random.seed!(random_state)
        println("Generating initial weight MPS with bond dimension χ = $χ_init
        using random state $random_state.")
    else
        println("Generating initial weight MPS with bond dimension χ = $χ_init.")
    end

    W = randomMPS(Complex{BigFloat},site_indices, linkdims=χ_init)

    label_idx = Index(num_classes, "f(x)")

    # get the site of interest and copy over the indices at the last site where we attach the label 
    old_site_idxs = inds(W[end])
    new_site_idxs = old_site_idxs, label_idx
    new_site = randomITensor(new_site_idxs)

    # add the new site back into the MPS
    W[end] = new_site

    # normalise the MPS
    normalize!(W)

    # canonicalise - bring MPS into canonical form by making all tensors 1,...,j-1 left orthogonal
    # here we assume we start at the right most index
    last_site = length(site_indices)
    orthogonalize!(W, last_site)

    return W

end


GenerateStartingMPS (generic function with 1 method)

In [27]:
random_state=123456
update_iters = 9
(X_train, y_train), (X_val, y_val), (X_test, y_test) = LoadSplitsFromTextFile("datasets/ECG_train.txt", 
    "datasets/ECG_val.txt", "datasets/ECG_test.txt")

X_train = vcat(X_train, X_val)
y_train = vcat(y_train, y_val)   
   
# first, create the site indices for the MPS and product states 
num_mps_sites = size(X_train)[2]
sites = siteinds("S=1/2", num_mps_sites)
println("Using χ_init=$χ_init and a maximum of $nsweep sweeps...")
println("Using $update_iters iterations per update.")

# now let's handle the training/validation/testing data
# rescale using a robust sigmoid transform
scaler = fitScaler(RobustSigmoidTransform, X_train; positive=true);
X_train_scaled = transformData(scaler, X_train)
X_val_scaled = transformData(scaler, X_val)
X_test_scaled = transformData(scaler, X_test)

# generate product states using rescaled data

training_states = GenerateAllProductStates(X_train_scaled, y_train, "train", sites)
validation_states = GenerateAllProductStates(X_val_scaled, y_val, "valid", sites)
testing_states = GenerateAllProductStates(X_test_scaled, y_test, "test", sites)

# generate the starting MPS with unfirom bond dimension χ_init and random values (with seed if provided)
num_classes = length(unique(y_train))
W = GenerateStartingMPS(χ_init, sites; num_classes=num_classes, random_state=random_state)


    ;

Using χ_init=5 and a maximum of nsweep sweeps...
Using 9 iterations per update.
Initialising train states.
Initialising valid states.
Initialising test states.
Generating initial weight MPS with bond dimension χ = 5
        using random state 123456.


In [28]:
phi = training_states[1]

yhat = ContractMPSAndProductState(W, phi)
label = phi.label # ground truth label
label_idx = inds(yhat)[1]
y = onehot(label_idx => label + 1) # one hot encode, so class 0 [1 0] is assigned using label_idx = 1
# compute the loss using the ground-truth y and model prediction yhat

ITensor ord=1 (dim=2|id=483|"f(x)")
NDTensors.Dense{Float64, Vector{Float64}}

In [45]:
p = phi.pstate[1]


ITensor ord=1 (dim=2|id=385|"S=1/2,Site,n=1")
NDTensors.Dense{Complex{BigFloat}, Vector{Complex{BigFloat}}}

In [47]:
pind = inds(p)

it = itensor(ComplexF64, NDTensors.array(p, pind), pind)

ITensor ord=1 (dim=2|id=385|"S=1/2,Site,n=1")
NDTensors.Dense{ComplexF64, Vector{ComplexF64}}

In [31]:
diff_sq = abs2.(yhat - y)
sum_of_sq_diff = real(sum(diff_sq))

loss = 0.5 * sum_of_sq_diff

# now get the predicted label
correct = 0

if (argmax(abs.(vector(yhat))) - 1) == ϕ.label
    correct = 1
end


UndefRefError: UndefRefError: access to undefined reference

In [16]:
ComputeLossPerSampleAndIsCorrect(W, training_states[1])

UndefRefError: UndefRefError: access to undefined reference

In [47]:
foo2(inds_c)

MethodError: MethodError: no method matching foo2(::NTuple{5, Index{Int64}})

Closest candidates are:
  foo2(!Matched::Tuple{Vararg{Index{Integer}}})
   @ Main ~/Documents/QML Project/QuantumInspiredML/MPS_MSE/testcache.ipynb:1


In [6]:
first(inds_c)

(dim=2|id=351|"C")

In [21]:
ls = eachslice(out; dims=1)

ls[1] = real(B_m)
ls[2] = imag(B_m);

In [25]:
it = inds(outIT)
l_ind = findindex(outIT, "C")

(dim=2|id=700|"C")

In [26]:
l_ind in it

true

In [30]:
findfirst(it .== l_ind)

1